<a href="https://colab.research.google.com/github/engineerJPark/FCN_Implementation/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

- transform 처리
- image 크기 안줄이고 1 batch로 처리
- dataset __getitem__() 수정



In [4]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset


# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import collections
import cv2
import PIL
from PIL import Image
import os
import shutil
import random
import xml.etree.ElementTree as ET


if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

torch.manual_seed(1)
if device == 'cuda:0':
    torch.cuda.manual_seed_all(1)

resnet_pretrained = resnet18(weights=ResNet18_Weights.DEFAULT)
# print(resnet_pretrained)

In [ ]:
# dataloader on PASCAL VOC. use name as train_set, val_set, test_set
# segmentation label 0 = background, 1 ~ 20 is object, 255 should be neglible

!mkdir ./data
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar -C /content/data

In [11]:
VOC_ROOT_DIR ="/content/data/VOCdevkit/VOC2012/"
ANNO_DIR = os.path.join(VOC_ROOT_DIR, "Annotations")
IMAGE_DIR = os.path.join(VOC_ROOT_DIR, "JPEGImages")

# xml_files = os.listdir(ANNO_DIR)
# print(xml_files[:5]); print(len(xml_files))

['2010_001535.xml', '2008_005101.xml', '2010_002660.xml', '2010_003632.xml', '2010_004848.xml']
17125


In [12]:
class VOCClassSegBase(torch.utils.data.Dataset):

    class_names = np.array([
        'background',
        'aeroplane',
        'bicycle',
        'bird',
        'boat',
        'bottle',
        'bus',
        'car',
        'cat',
        'chair',
        'cow',
        'diningtable',
        'dog',
        'horse',
        'motorbike',
        'person',
        'potted plant',
        'sheep',
        'sofa',
        'train',
        'tv/monitor',
    ])
    mean_bgr = np.array([104.00698793, 116.66876762, 122.67891434])

    def __init__(self, root='/content/data', split='train', transform=None):
        self.root = root
        self.split = split
        self.transform = transform

        dataset_dir = os.path.join(self.root, 'VOCdevkit/VOC2012')
        self.files = collections.defaultdict(list)
        for split in ['train', 'val']:
            imgsets_file = os.path.join(
                dataset_dir, 'ImageSets/Segmentation/%s.txt' % split)
            for did in open(imgsets_file):
                did = did.strip()
                img_file = os.path.join(dataset_dir, 'JPEGImages/%s.jpg' % did)
                lbl_file = os.path.join(
                    dataset_dir, 'SegmentationClass/%s.png' % did)
                self.files[split].append({
                    'img': img_file,
                    'lbl': lbl_file,
                })

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        data_file = self.files[self.split][index]
        
        # load image
        img_file = data_file['img']
        img = PIL.Image.open(img_file)
        img = np.array(img, dtype=np.uint8)

        # load label
        lbl_file = data_file['lbl']
        lbl = PIL.Image.open(lbl_file)
        lbl = np.array(lbl, dtype=np.int32)

        # image preprocessing
        img = img[:, :, ::-1]  # BGR -> RGB
        img = img.astype(np.float64)
        img -= self.mean_bgr
        # img = img.transpose(2, 0, 1) # HWC -> CHW
        # img = torch.from_numpy(img).float()
        print('image size :', img.shape)
        # label preprocessing
        lbl[lbl == 255] = -1.
        # lbl = torch.from_numpy(lbl).long()
        
        if self.transform is not None:
            return self.transform(img), lbl
        else:
            return img, lbl

In [20]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256,256)),
    # transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

# # test
# sample_img = Image.open('/content/data/VOCdevkit/VOC2012/JPEGImages/2007_000032.jpg')

# plt.figure(figsize=(8, 8))
# plt.imshow(sample_img)
# plt.show()

# transformed_img = transform(sample_img)

# plt.figure(figsize=(8, 8))
# plt.imshow(transformed_img.permute(1,2,0))
# plt.show()

In [21]:
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
# https://blog.promedius.ai/pytorch_dataloader_1/

# train_data = torchvision.datasets.ImageFolder(root="/content/train_data", transform=transform)
# train_gt_data = torchvision.datasets.ImageFolder(root="/content/train_gt_data", transform=transform)

train_data = VOCClassSegBase(root='/content/data', split='train', transform=transform)

train_data_loader = DataLoader(dataset=train_data, batch_size = 1,
                               drop_last=True)

In [15]:
# Bilinear weights deconvolution Algorithm 알아보기
def bilinear_kernel_init(Cin, Cout, kernel_size):
  factor = (kernel_size + 1) // 2
  if kernel_size % 2 == 1:
    center = (kernel_size + 1) // 2 - 1
  else:
    center = (kernel_size + 1) // 2 - 0.5

  og = (torch.arange(kernel_size).reshape(-1,1), torch.arange(kernel_size).reshape(1,-1))
  filter = (1 - torch.abs(og[0] - center) / factor) * (1 - torch.abs(og[1] - center) / factor)

  weight = torch.zeros((Cin, Cout, kernel_size, kernel_size))
  weight[range(Cin), range(Cout), :, :] = filter
  return weight

In [16]:
class FCN18(nn.Module):
  def __init__(self, class_n): # class 20 + 1(bakcground)
    super().__init__()
    self.downsample1 = nn.Sequential( # 1/2 of the input
        resnet_pretrained.conv1,
        resnet_pretrained.bn1,
        resnet_pretrained.relu,
        resnet_pretrained.maxpool
    )
    self.downsample2 = resnet_pretrained.layer1 # 1/4 of the input
    self.downsample3 = resnet_pretrained.layer2 # 1/8 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample4 = resnet_pretrained.layer3 # 1/16 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample5 = resnet_pretrained.layer4 # 1/32 of the input, 여기서 1x1 거친것 하나 추출

    self.fully_conv_pool3 = nn.Conv2d(128, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool3.weight)

    self.fully_conv_pool4 = nn.Conv2d(256, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool4.weight)

    self.fully_conv_pool5 = nn.Conv2d(512, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool5.weight)
    
    # 보통 stride s, padding s/2, kernelsize 2s -> s배 만큼 이미지가 커진다.
    self.upsample_make_16s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/16
    self.upsample_make_16s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))

    self.upsample_make_8s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/8
    self.upsample_make_8s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))
    
    self.upsample_to_score = nn.ConvTranspose2d(class_n, class_n, kernel_size=16, padding=4, stride=8) # to 1
    self.upsample_to_score.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 16))
    for param in self.upsample_to_score.parameters(): # freeze the last layer
      param.requires_grad = False


  def forward(self, x):
    x = self.downsample1(x)
    x = self.downsample2(x)
    x = self.downsample3(x)
    pool3_score = self.fully_conv_pool3(x)
    x = self.downsample4(x)
    pool4_score = self.fully_conv_pool4(x)
    x = self.downsample5(x)
    pool5_score = self.fully_conv_pool5(x)

    pool4_2x_conv7 = pool4_score + self.upsample_make_16s(pool5_score)
    pool3_2x_pool4_4x_conv7 = pool3_score + self.upsample_make_8s(pool4_2x_conv7)

    out = self.upsample_to_score(pool3_2x_pool4_4x_conv7)

    return out

In [17]:
epochs = 10
lr = 10e-4
weight_decay = 2e-4
momentum = 0.9
batch_size = 64

model = FCN18(3)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss(ignore_index= 255)

In [23]:
# train code
def train(model, epochs, optimizer, criterion):
  for epoch in range(epochs):
    for train_img, train_gt_img in train_data_loader:
      N, C, H, W = train_img.shape
      print('train_img.shape:', N, C, H, W)
      print('train_gt_img.shape:', train_gt_img.shape)
      # print(train_gt_img[0][0][150][150])

      N_t, H_t, W_t = train_gt_img.shape
      
      train_img = train_img.to(device)
      train_gt_img = train_gt_img.to(device)

      score_img = model(train_img.to(torch.float))
      score_img.permute(0,2,3,1).reshape(N*H*W, C)
      train_gt_img.reshape(N_t*H_t*W_t,)

      loss = criterion(score_img, train_gt_img)

      optimizer.zero_grad()
      optimizer.step()
      loss.backward()

    print("epoch %d, loss : %f "%(epoch + 1, loss))
  print("Training End")

train(model, epochs, optimizer, criterion)

image size : (281, 500, 3)
train_img.shape: 1 3 256 256
train_gt_img.shape: torch.Size([1, 281, 500])


RuntimeError: ignored

In [ ]:
# segmentation test code 

with open("/content/data/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt", 'r') as f:
  lines = f.readlines()
for i in range(len(lines)):
  lines[i] =  lines[i].strip('\n')
test_jpg = lines[0] + '.jpg'
test_image = Image.open(os.path.join(VOC_ROOT_DIR, "JPEGImages", test_jpg))

test_seg = model(test_image)

plt.figure(figsize=(8, 8))
plt.imshow(test_seg)
plt.show()

In [ ]:
# validation



In [ ]:
# visualization

